# Assignment 2: Naive Bayes and Text Classification

Only use the already imported library `numpy`. Make sure that the `spamham.txt` dataset is in the same directory as the notebook.

List your team members (name and immatriculation number) in the following cell:

- *Kuang Yu Li xxxxxxx*
- *Ya-Jen Hsu 3449448*
- *Gabriella Ilena 3440942*

In [22]:
# Load required packages and dataset. Do not modify.
import numpy as np


def load_spamham_dataset():
    import string
    
    with open('spamham.txt', mode='r', encoding='utf-8') as f:
        rows = [l.strip().split('\t')[:2] for l in f]
    
    y, X = zip(*rows)
    X =[x.translate(str.maketrans('', '', string.punctuation)).lower().split() for x in X]
    
    return X, y
    

X, y = load_spamham_dataset()

print('Sample:')
print(f'{y[0]}: {X[0]}')
print(f'{y[2]}: {X[2]}')

Sample:
ham: ['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'there', 'got', 'amore', 'wat']
spam: ['free', 'entry', 'in', '2', 'a', 'wkly', 'comp', 'to', 'win', 'fa', 'cup', 'final', 'tkts', '21st', 'may', '2005', 'text', 'fa', 'to', '87121', 'to', 'receive', 'entry', 'questionstd', 'txt', 'ratetcs', 'apply', '08452810075over18s']


## Task 2: Spam Classification with Naive Bayes

Check out the description of the dataset at [https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection](https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection).

Implement a Naive Bayes classifier with Laplace smoothing to detect whether a text message is spam or ham (not spam).

A text message is represented by a list of string tokens as shown above.
The classification target is binary and the two possible labels are the strings `'spam'` and `'ham'`.

Fill out the methods in `NaiveBayesSpamClassifier` to train (`fit`) and predict (`predict`). Feel free to introduce new fields and methods based on your needs, but the methods `fit` and `predict` are required and their interface should not be changed.

Hint: Try to map the text messages to word frequency vectors by counting how often each word occurs in a message.

In [23]:
# Implement your solution here.
class NaiveBayesSpamClassifier(object):
    def __init__(self):
        pass
    
    def fit(self, X, y):
        """
        X is a list of `n` text messages. Each text message is a list of strings with at least length one.
        y is a list of `n` labels either the string 'spam' or the string 'ham'.
        """
        # Calculate priors based on the class labels in y
        label_count = {'spam': 0, 'ham': 0}
        for label in y:
            if type(label) == str and label == 'spam': # Checking the type just in case
                label_count['spam'] += 1
            elif type(label) == str and label == 'ham':
                label_count['ham'] += 1
        prior_spam = label_count['spam'] / len(y) # Number of spam over all elements
        prior_ham = label_count['ham'] / len(y) # Number of ham over all elements
        priors = [prior_spam, prior_ham]

        # Printing for testing purposes only
        print(label_count)
        print(priors)
        assert sum(label_count.values()) == len(y)

        # Create a bag of words for each class label and count the word frequency
        # Add to global vocabulary
        bag_words = {'spam':{}, 'ham':{}}

        for x, y in zip(X, y):

            word_count = {}
            # Go through each word in current class label
            for word in x:
                word_count[word] = word_count.get(word, 0) + 1

            # Iterate over the dictionary
            for word, count in word_count.items():
                # If the word is not in this class
                if word not in bag_words[y]:
                    bag_words[y][word] = 0
                # Add the words to relevant bag of words
                bag_words[y][word] += count
        
        # Print to test result
        print(bag_words)
        # pass
    
    def predict(self, X):
        """
        X is a list of `n` text messages. Each text message is a list of strings with at least length one.
        The method returns a list of `n` strings, i.e. classification labels ('spam' or 'ham').
        """
        return None

# Testing
mytest = NaiveBayesSpamClassifier()
mytest.fit(X, y)

{'spam': 747, 'ham': 4827}
[0.1340150699677072, 0.8659849300322928]
{'spam': {'free': 216, 'entry': 26, 'in': 73, '2': 173, 'a': 378, 'wkly': 14, 'comp': 10, 'to': 689, 'win': 60, 'fa': 4, 'cup': 5, 'final': 16, 'tkts': 4, '21st': 2, 'may': 7, '2005': 3, 'text': 120, '87121': 4, 'receive': 33, 'questionstd': 2, 'txt': 150, 'ratetcs': 2, 'apply': 29, '08452810075over18s': 2, 'freemsg': 12, 'hey': 5, 'there': 11, 'darling': 2, 'its': 11, 'been': 43, '3': 22, 'weeks': 13, 'now': 189, 'and': 122, 'no': 60, 'word': 22, 'back': 23, 'id': 3, 'like': 13, 'some': 6, 'fun': 9, 'you': 287, 'up': 21, 'for': 203, 'it': 28, 'still': 7, 'tb': 1, 'ok': 5, 'xxx': 11, 'std': 9, 'chgs': 1, 'send': 68, '£150': 27, 'rcv': 2, 'winner': 16, 'as': 34, 'valued': 11, 'network': 25, 'customer': 49, 'have': 135, 'selected': 25, 'receivea': 2, '£900': 7, 'prize': 92, 'reward': 10, 'claim': 113, 'call': 347, '09061701461': 2, 'code': 29, 'kl341': 2, 'valid': 23, '12': 10, 'hours': 5, 'only': 79, 'had': 18, 'your': 

In [24]:
# The following code will evaluate your classifier.
class HamClassifier(object):
    """
    This classifier is a primitive baseline, which just predicts the most common class each time.
    Naive Bayes should definitely beat this.
    """
    def fit(self, X, y): pass
    def predict(self, X): return len(X)*['ham']

    
def train_evaluate(classifier, X, y):
    from sklearn.metrics import confusion_matrix
    from sklearn.model_selection import train_test_split
    
    # Apply train-test split.
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2020)
    # Inititialize and train classifier.
    classifier.fit(X_train, y_train)
    # Evaluate classifier on test data.
    yhat_test = classifier.predict(X_test)
    cmatrix = confusion_matrix(y_test, yhat_test, labels=['ham', 'spam'])
    
    return cmatrix


def plot_confusion_matrix(cmatrix, classifier_name):
    import matplotlib.pyplot as plt
    
    fig, ax = plt.subplots(1, 1)
    ax.matshow(cmatrix, cmap='Greens')
    for x in (0, 1):
        for y in (0, 1):
            ax.text(x, y, cmatrix[y, x])
    ax.set_xlabel('predicted label')
    ax.set_ylabel('true label')
    ax.set_xticklabels(['', 'ham', 'spam'])
    ax.set_yticklabels(['', 'ham', 'spam'])
    ax.set_title(classifier_name)

    
    
ham_classifier = HamClassifier()
your_classifier = NaiveBayesSpamClassifier()
ham_cmatrix = train_evaluate(ham_classifier, X, y)
your_cmatrix = train_evaluate(your_classifier, X, y)

plot_confusion_matrix(ham_cmatrix, 'HamClassifier')
plot_confusion_matrix(your_cmatrix, 'NaiveBayesSpamClassifier')

{'spam': 519, 'ham': 3382}
[0.1330428095360164, 0.8669571904639836]
{'spam': {'private': 14, 'your': 186, '2003': 10, 'account': 12, 'statement': 12, 'for': 157, '07753741225': 1, 'shows': 22, '800': 9, 'unredeemed': 11, 's': 9, 'i': 37, 'm': 8, 'points': 11, 'call': 230, '08715203677': 1, 'identifier': 11, 'code': 19, '42478': 1, 'expires': 12, '241004': 1, 'freemessage': 1, 'jamsterget': 1, 'the': 139, 'crazy': 4, 'frog': 1, 'sound': 1, 'now': 124, 'poly': 14, 'text': 85, 'mad1': 1, 'real': 8, 'mad2': 1, 'to': 487, '88888': 2, '6': 4, 'sounds': 1, 'just': 46, '3': 17, 'gbpweek': 1, '16only': 2, 'tcs': 20, 'apply': 23, 'you': 202, 'have': 92, 'won': 50, 'a': 265, 'guaranteed': 33, '£1000': 28, 'cash': 45, 'or': 124, '£2000': 20, 'prize': 63, 'claim': 74, 'yr': 6, 'our': 52, 'customer': 35, 'service': 35, 'representative': 7, 'on': 105, '08714712379': 1, 'between': 7, '10am7pm': 3, 'cost': 17, '10p': 8, 'records': 4, 'indicate': 2, 'u': 98, 'maybe': 2, 'entitled': 5, '5000': 3, 'pounds

ValueError: Expected array-like (array or non-string sequence), got None